# calibration

> Funtions related to calibration of LR.

In [ ]:
#| default_exp calibration

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

def get_filtered_images(filters, 
                        face_image_filters,
                        filter_values: tuple,
                        detector,
                        embeddingModel,
                        calibration_db,
                        session):


    im_filter_values = filter_values[:len(filters)]
    fi_filter_values = filter_values[len(filters):]
    assert len(fi_filter_values) == len(face_image_filters)

    query = session.query(FaceImage, Image.identity, Image.image_id)
    join_query = query \
        .join(CroppedImage, CroppedImage.croppedImage_id == FaceImage.croppedImage_id) \
        .join(Image, Image.image_id == CroppedImage.image_id) \
        .join(Detector) \
        .join(EmbeddingModel)
    filter_query = join_query \
        .filter(EmbeddingModel.name == embeddingModel,
                Detector.name == detector) \
        .filter(Image.source.in_(calibration_db))
    for cal_filter, value in zip(face_image_filters, fi_filter_values):
        filter_query = filter_query.filter(FaceImage.__dict__[cal_filter] == value)
    for cal_filter, value in zip(filters, im_filter_values):
        filter_query = filter_query.filter(Image.__dict__[cal_filter] == value)
    return filter_query.all()

In [ ]:
#| export

def get_calibration_pairs_per_category(self, categories, session):
        cal_face_pairs = {}
        emb_facevacs = (self.embeddingModel == 'FaceVACs')

        for pair_category in categories:

            first_image_category = self.get_filtered_images(filter_values=pair_category[0], session=session)
            if pair_category[0] == pair_category[1]:
                if emb_facevacs:
                    all_calibration_pairs = get_calibration_facepairs_facevacs(
                        first_list_of_face_images=first_image_category,
                        second_list_of_face_images=first_image_category,
                        number_of_pairs=self.n_calibration_pairs,
                        session=session
                    )
                else:
                    all_calibration_pairs = make_cal_face_pairs(first_list_of_face_images=first_image_category,
                                                                number_of_pairs=self.n_calibration_pairs)

            else:
                second_image_category = self.get_filtered_images(filter_values=pair_category[1],
                                                                 session=session)
                if emb_facevacs:
                    all_calibration_pairs = get_calibration_facepairs_facevacs(
                        first_list_of_face_images=first_image_category,
                        second_list_of_face_images=second_image_category,
                        number_of_pairs=self.n_calibration_pairs,
                        session=session
                    )
                else:
                    all_calibration_pairs = make_cal_face_pairs(first_list_of_face_images=first_image_category,
                                                                second_list_of_face_images=second_image_category,
                                                                number_of_pairs=self.n_calibration_pairs)
            cal_face_pairs[pair_category] = all_calibration_pairs
        return cal_face_pairs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()